# 1. Setup Mario

In [1]:
!pip install gym_super_mario_bros==7.3.0 nes_py

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Game
import gym_super_mario_bros
# Joypad
from nes_py.wrappers import JoypadSpace
# Controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [3]:
SIMPLE_MOVEMENT

[['NOOP'],
 ['right'],
 ['right', 'A'],
 ['right', 'B'],
 ['right', 'A', 'B'],
 ['A'],
 ['left']]

In [4]:
# Setup game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# Simplify movement variations (action_space)
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [5]:
# Flag
done = True

# Loop through each frame
for step in range(100):
    if done:
        # Start the game
        env.reset()
    # Do random actions
    state, reward, done, info = env.step(env.action_space.sample())
    # Show the game on the screen
    env.render()
env.close()

C:\Users\yacoo\AppData\Roaming\Python\Python310\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


# 2. Preprocess Environment

In [6]:
!pip install torch torchvision torchaudio
# PPO for RL
!pip install stable-baselines3[extra]

Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


In [7]:
# Frame Stacker Wrapper and Grayscaling Wrapper
# Framestack: Capture last couple of frames while Mario is playing
# GrayscaleObservation: Shave our data since it does not deal with no coloring, making our model faster
from gym.wrappers import GrayScaleObservation
# Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Plot the impact of framestacking
from matplotlib import pyplot as plt

In [8]:
# Create the base env 
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# Simplify movement controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# Grayscale the env
env = GrayScaleObservation(env, keep_dim=True)
# Wrap inside the dummy env
env = DummyVecEnv([lambda: env])
# Stack the frames
# Find how many frames to take in as you desire that works best
env = VecFrameStack(env, 4, channels_order='last')

In [9]:
# state = env.reset()
# plt.imshow(state)

In [23]:
env.reset()

array([[[[  0,   0,   0, 140],
         [  0,   0,   0, 140],
         [  0,   0,   0, 140],
         ...,
         [  0,   0,   0, 140],
         [  0,   0,   0, 140],
         [  0,   0,   0, 140]],

        [[  0,   0,   0, 140],
         [  0,   0,   0, 140],
         [  0,   0,   0, 140],
         ...,
         [  0,   0,   0, 140],
         [  0,   0,   0, 140],
         [  0,   0,   0, 140]],

        [[  0,   0,   0, 140],
         [  0,   0,   0, 140],
         [  0,   0,   0, 140],
         ...,
         [  0,   0,   0, 140],
         [  0,   0,   0, 140],
         [  0,   0,   0, 140]],

        ...,

        [[  0,   0,   0, 214],
         [  0,   0,   0, 124],
         [  0,   0,   0, 124],
         ...,
         [  0,   0,   0, 124],
         [  0,   0,   0, 124],
         [  0,   0,   0,   0]],

        [[  0,   0,   0, 214],
         [  0,   0,   0, 124],
         [  0,   0,   0, 124],
         ...,
         [  0,   0,   0, 124],
         [  0,   0,   0,   0],
         

In [24]:
state, reward, done, info = env.step([5])

In [12]:
#plt.figure(figsize=(20,16))
#for idx in range(state.shape[3]):
 #   plt.subplot(1,4,idx+1)
 #   plt.imshow(state[0][:,:,idx])
#plt.show()

# 3. Train the RL Model

In [9]:
# AREA: Agent, Reward, Environment, Action
# PPO: Proximal Policy Optimization
# OS for file path management
import os
# PPO classifier
from stable_baselines3 import PPO
# Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [10]:
# Save our model every x number of steps
class TrainAndLoggingCallback(BaseCallback):
    
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
        
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
            
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
            
        return True

In [11]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [12]:
callback = TrainAndLoggingCallback(check_freq=1000000, save_path=CHECKPOINT_DIR)

In [20]:
# RL requires a policy network (such as the CNN that is the brain of the AI)
# CnnPolicy is fast at processing images, which fits our use case
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [18]:
# Train the model
# Every single game gets a million frames
#model.learn(total_timesteps=4000000, callback=callback)

In [19]:
# No callback: model.save('name_of_file')
# To view the logs on tensorboard
# Activate the env on this dir and go into log dir (go into most recent ppo model)
#tensorboard --logdir=.

# 4. Test the Model

In [25]:
# Load the model
model = PPO.load('./train/best_model_1000000', env)

Wrapping the env in a VecTransposeImage.


In [ ]:
# Continue training the model
model.learn(total_timesteps=4000000, callback=callback)

Logging to ./logs/PPO_12
----------------------------
| time/              |     |
|    fps             | 94  |
|    iterations      | 1   |
|    time_elapsed    | 5   |
|    total_timesteps | 512 |
----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 14           |
|    iterations           | 2            |
|    time_elapsed         | 69           |
|    total_timesteps      | 1024         |
| train/                  |              |
|    approx_kl            | 0.0006724793 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.263       |
|    explained_variance   | 0.907        |
|    learning_rate        | 1e-06        |
|    loss                 | 65.3         |
|    n_updates            | 19540        |
|    policy_gradient_loss | -0.000268    |
|    value_loss           | 157          |
-----------------------------------------

-------------------------------------------
| time/                   |               |
|    fps                  | 8             |
|    iterations           | 13            |
|    time_elapsed         | 796           |
|    total_timesteps      | 6656          |
| train/                  |               |
|    approx_kl            | 1.0414282e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.329        |
|    explained_variance   | 0.732         |
|    learning_rate        | 1e-06         |
|    loss                 | 332           |
|    n_updates            | 19650         |
|    policy_gradient_loss | 0.000355      |
|    value_loss           | 630           |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 8            |
|    iterations           | 14           |
|    time_elapsed         | 863     

------------------------------------------
| time/                   |              |
|    fps                  | 8            |
|    iterations           | 24           |
|    time_elapsed         | 1498         |
|    total_timesteps      | 12288        |
| train/                  |              |
|    approx_kl            | 0.0012308394 |
|    clip_fraction        | 0.00117      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.279       |
|    explained_variance   | 0.949        |
|    learning_rate        | 1e-06        |
|    loss                 | 44.7         |
|    n_updates            | 19760        |
|    policy_gradient_loss | -0.000854    |
|    value_loss           | 121          |
------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 8             |
|    iterations           | 25            |
|    time_elapsed         | 1560          |
|    t

-------------------------------------------
| time/                   |               |
|    fps                  | 8             |
|    iterations           | 35            |
|    time_elapsed         | 2175          |
|    total_timesteps      | 17920         |
| train/                  |               |
|    approx_kl            | 0.00064398337 |
|    clip_fraction        | 0.000781      |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.629        |
|    explained_variance   | 0.794         |
|    learning_rate        | 1e-06         |
|    loss                 | 329           |
|    n_updates            | 19870         |
|    policy_gradient_loss | -0.000659     |
|    value_loss           | 837           |
-------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 8           |
|    iterations           | 36          |
|    time_elapsed         | 2232        

-------------------------------------------
| time/                   |               |
|    fps                  | 8             |
|    iterations           | 46            |
|    time_elapsed         | 2831          |
|    total_timesteps      | 23552         |
| train/                  |               |
|    approx_kl            | 0.00088433805 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.55         |
|    explained_variance   | -0.589        |
|    learning_rate        | 1e-06         |
|    loss                 | 0.13          |
|    n_updates            | 19980         |
|    policy_gradient_loss | -0.00173      |
|    value_loss           | 0.321         |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 8            |
|    iterations           | 47           |
|    time_elapsed         | 2897    

...

------------------------------------------
| time/                   |              |
|    fps                  | 8            |
|    iterations           | 673          |
|    time_elapsed         | 41579        |
|    total_timesteps      | 344576       |
| train/                  |              |
|    approx_kl            | 0.0018799922 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.849       |
|    explained_variance   | 0.533        |
|    learning_rate        | 1e-06        |
|    loss                 | 99.4         |
|    n_updates            | 26250        |
|    policy_gradient_loss | -0.00146     |
|    value_loss           | 226          |
------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 8           |
|    iterations           | 674         |
|    time_elapsed         | 41642       |
|    total_times

------------------------------------------
| time/                   |              |
|    fps                  | 8            |
|    iterations           | 684          |
|    time_elapsed         | 42242        |
|    total_timesteps      | 350208       |
| train/                  |              |
|    approx_kl            | 0.0059566777 |
|    clip_fraction        | 0.000586     |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.945       |
|    explained_variance   | 0.785        |
|    learning_rate        | 1e-06        |
|    loss                 | 55.1         |
|    n_updates            | 26360        |
|    policy_gradient_loss | -0.00184     |
|    value_loss           | 423          |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 8            |
|    iterations           | 685          |
|    time_elapsed         | 42297        |
|    total_

-------------------------------------------
| time/                   |               |
|    fps                  | 8             |
|    iterations           | 695           |
|    time_elapsed         | 42882         |
|    total_timesteps      | 355840        |
| train/                  |               |
|    approx_kl            | 0.00012454728 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.887        |
|    explained_variance   | 0.836         |
|    learning_rate        | 1e-06         |
|    loss                 | 133           |
|    n_updates            | 26470         |
|    policy_gradient_loss | 3.67e-06      |
|    value_loss           | 365           |
-------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 8           |
|    iterations           | 696         |
|    time_elapsed         | 42945       

-------------------------------------------
| time/                   |               |
|    fps                  | 8             |
|    iterations           | 706           |
|    time_elapsed         | 43565         |
|    total_timesteps      | 361472        |
| train/                  |               |
|    approx_kl            | 0.00067213527 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.908        |
|    explained_variance   | 0.852         |
|    learning_rate        | 1e-06         |
|    loss                 | 67.4          |
|    n_updates            | 26580         |
|    policy_gradient_loss | -9.04e-05     |
|    value_loss           | 195           |
-------------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 8          |
|    iterations           | 707        |
|    time_elapsed         | 43627      |
|  

-----------------------------------------
| time/                   |             |
|    fps                  | 8           |
|    iterations           | 717         |
|    time_elapsed         | 44251       |
|    total_timesteps      | 367104      |
| train/                  |             |
|    approx_kl            | 0.011897487 |
|    clip_fraction        | 0.024       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.919      |
|    explained_variance   | 0.717       |
|    learning_rate        | 1e-06       |
|    loss                 | 81.2        |
|    n_updates            | 26690       |
|    policy_gradient_loss | -0.00588    |
|    value_loss           | 201         |
-----------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 8            |
|    iterations           | 718          |
|    time_elapsed         | 44314        |
|    total_timesteps      | 3

In [ ]:
#state = env.reset()

#while True:
 #   action, _ = model.predict(state)
 #   state, reward, done, info = env.step(action)
 #   env.render()